In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split #for splitting the data in train and test
from sklearn.preprocessing import MinMaxScaler #for various scaling methods

from sklearn.metrics import accuracy_score,confusion_matrix,recall_score #for accuracy matrices
from sklearn.metrics import precision_score,classification_report,roc_auc_score,precision_score #for accuracy matrices
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/ZikaVirus/classification_dataset.csv')
df.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Target
0,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
1,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
2,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
3,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
4,12625.800781,-34.603684,-58.381559,22,10,0.0,11.5,76,10,27,34,72,0.0,1025,10,99,2016,3,0


In [6]:
df.duplicated().sum()

79878

In [7]:
df = df.drop_duplicates(keep='first', inplace=False)

In [8]:
df.shape

(75345, 19)

In [9]:
from sklearn.preprocessing import StandardScaler
feature_scale = [feature for feature in df.columns if feature not in ['Target']]

scaler=StandardScaler()
scaler.fit(df[feature_scale])

# transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([df[['Target']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(df[feature_scale]), columns=feature_scale)],axis=1)

data.head()

,Target,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month
0,1,5.006081,-3.696627,2.018093,-0.440424,-1.502043,-0.0376,0.229937,1.348236,-1.541039,0.941944,-1.680504,-1.072223,-0.631531,1.521908,0.857886,0.161154,-0.364742,-0.890727
1,0,5.006081,-3.696627,2.018093,-0.440424,-1.502043,-0.0376,0.229937,1.348236,-1.541039,0.941944,-1.680504,-1.072223,-0.631531,1.521908,0.857886,0.161154,-0.364742,-0.890727
2,0,5.006081,-3.696627,2.018093,-0.738510,-1.035319,-0.0376,0.169162,0.995244,-0.884426,2.565979,-0.004024,-0.113112,-0.631531,3.739487,0.857886,-1.253408,-0.364742,-0.890727
3,1,5.006081,-3.696627,2.018093,-0.738510,-1.035319,-0.0376,0.169162,0.995244,-0.884426,2.565979,-0.004024,-0.113112,-0.631531,3.739487,0.857886,-1.253408,-0.364742,-0.890727
4,0,5.006081,-3.696627,2.018093,-0.738510,0.209279,-0.0376,-1.593318,-0.609263,-0.063659,1.089584,1.327298,0.329554,2.084641,0.571517,-0.632623,-1.159104,-0.364742,-0.564818


In [10]:
X = data.drop('Target',axis=1)
y = data['Target']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=10, stratify=y) #train test split in 50:50 ratio
X_train.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month
9747,-0.290158,-0.250896,0.430378,-1.483724,-1.190893,-0.0376,0.229937,-0.352542,-0.884426,-0.091533,0.636983,0.919776,-0.459206,-0.062077,-2.123131,-0.357519,-0.364742,-1.216636
26293,-0.319180,-0.450552,0.335489,-0.589467,-0.101871,-0.0376,-0.438589,0.096720,0.428801,-0.534451,1.080757,1.141109,2.683676,0.571517,-1.377877,1.324238,-0.364742,-0.238909
41021,-0.269929,0.247651,0.556984,1.050003,1.453877,-0.0376,0.229937,1.155695,1.085414,2.123060,-0.792956,-0.629556,-0.426382,-1.329265,0.857886,-1.520603,-0.364742,0.738818
12705,-0.306874,-0.335913,0.140681,-0.142339,-0.101871,-0.0376,0.229937,-0.994345,-0.063659,-0.386812,-0.645031,0.108221,0.139829,-0.062077,0.112631,0.538370,-0.364742,-0.890727
48883,-0.317182,-0.303278,0.097710,-0.440424,-0.568595,-0.0376,0.229937,-1.379427,-0.391966,-0.977370,-0.053332,-0.113112,0.221888,0.254720,0.112631,-0.577562,-0.364742,1.390636


In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((37672, 18), (37672,), (37673, 18), (37673,))

## Hyperparameter tuning

In [12]:
import tensorflow.keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU
from tensorflow.keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU


In [13]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=2)

In [14]:
model

In [15]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[10])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [16]:
grid_result = grid.fit(X_train, y_train)

Epoch 1/10
236/236 - 1s - loss: 0.7085 - accuracy: 0.5526
Epoch 2/10
236/236 - 0s - loss: 0.6800 - accuracy: 0.5677
Epoch 3/10
236/236 - 0s - loss: 0.6749 - accuracy: 0.5695
Epoch 4/10
236/236 - 0s - loss: 0.6736 - accuracy: 0.5713
Epoch 5/10
236/236 - 0s - loss: 0.6721 - accuracy: 0.5715
Epoch 6/10
236/236 - 0s - loss: 0.6716 - accuracy: 0.5735
Epoch 7/10
236/236 - 0s - loss: 0.6702 - accuracy: 0.5746
Epoch 8/10
236/236 - 0s - loss: 0.6692 - accuracy: 0.5779
Epoch 9/10
236/236 - 0s - loss: 0.6688 - accuracy: 0.5759
Epoch 10/10
236/236 - 0s - loss: 0.6683 - accuracy: 0.5769
59/59 - 0s - loss: 0.6679 - accuracy: 0.5728
Epoch 1/10
236/236 - 1s - loss: 0.9174 - accuracy: 0.5081
Epoch 2/10
236/236 - 0s - loss: 0.7122 - accuracy: 0.5339
Epoch 3/10
236/236 - 0s - loss: 0.6921 - accuracy: 0.5543
Epoch 4/10
236/236 - 0s - loss: 0.6851 - accuracy: 0.5639
Epoch 5/10
236/236 - 0s - loss: 0.6806 - accuracy: 0.5681
Epoch 6/10
236/236 - 0s - loss: 0.6779 - accuracy: 0.5705
Epoch 7/10
236/236 - 0s - 

In [17]:
[grid_result.best_score_,grid_result.best_params_]

[0.5765819191932678,
 {'activation': 'relu',
  'batch_size': 256,
  'epochs': 10,
  'layers': [45, 30, 15]}]

In [18]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[13740,  6627],
       [ 9151,  8155]])

In [20]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score

0.5811854643909431

In [23]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.0 MB/s 


# Hyperparameter tuning

In [24]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 60)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [26]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='projectzika',
    project_name='ZikaVirus')

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 60, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [28]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

In [30]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

Trial 6 Complete [00h 26m 12s]
val_accuracy: 0.539352297782898

Best val_accuracy So Far: 0.5913575490315756
Total elapsed time: 01h 39m 22s
INFO:tensorflow:Oracle triggered exit


In [32]:
tuner.results_summary()

Results summary
Results in projectzika/ZikaVirus
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 29
units_0: 480
units_1: 64
learning_rate: 0.0001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
units_20: 32
units_21: 32
units_22: 32
units_23: 32
units_24: 32
units_25: 32
units_26: 32
units_27: 32
units_28: 32
Score: 0.5913575490315756
Trial summary
Hyperparameters:
num_layers: 36
units_0: 64
units_1: 288
learning_rate: 0.01
units_2: 416
units_3: 128
units_4: 352
units_5: 416
units_6: 416
units_7: 352
units_8: 192
units_9: 480
units_10: 224
units_11: 352
units_12: 256
units_13: 32
units_14: 288
units_15: 384
units_16: 64
units_17: 160
units_18: 128
units_19: 320
units_20: 96
units_21: 256
units_22: 32
units_23: 448
units_24: 320
units_25: 416
uni